## Thinking Outside the Bot #1: Reinforcement Learning Basics

## Introduction

In my free time I enjoy experimenting with AI.  Not just solving a problem but playing around with models and parameters to get a feel for what's really going on.  One of the topics I find particularly interesting is reinforcement learning (RL).  Instead of fitting a model to a dataset, RL forces us to be creative in designing an agent that experiences the data in the form of the game state and labels in the form of rewards.  Therefore, not only do we have to be careful in how we design the model, but we need to consider how we construct the game, inputs, and reward system.  However, despite the freedom inherent in reinforcement learning, there exists an underlying structure to each problem that the model is trying to learn.  Hopefully, during the course of this notebook I'll successfully illustrate both the structure behind RL as well as the methodology behind reinforcement learning as a whole.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/sample_random.gif?raw=true)

## Cartpole

In order to best discuss the core concepts behind RL, I've decided to use the simplest use case known as the cartpole problem.  It consists of a pole standing on top of a cart.  The player can accelerate the cart right or left, moving the pivot point in an attempt to prevent the pole from falling.  The model is fed four different inputs; the position of the cart ($x$), the velocity of the cart ($\dot x$), the angle of the pole ($\theta$), and the angular velocity of the pole ($\dot \theta$).

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/ObservationSpace.PNG?raw=true)

At the beginning of the game, each of the four values are randomly initialized between $-0.05$ and $0.05$ so that each initial state is slightly different but close to zero.  The game ends when either the absolute value of the x position exceeds $2.4$, the pole angle exceeds $41.8^{\circ}$ or $0.2$ radians, or the game lasts for $500$ frames.  The limitation on the cart is a lot more lenient than the pole, which means any agent training to balance the pole will prioritize the pole's position over the cart's.  The agent has two options; accelerate the cart left or right.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/ActionSpace.PNG?raw=true)

Note that the agent is not allowed to keep the current speed but must accelerate in one of the directions.  The agent is also not allowed to directly adjust the pole speed.  Instead it must accelerate the cart and use the resultant torque to keep the pole standing up.

## Physics

Before I get into the models I've used, let's talk about the environment and the physics behind it.  If you're not interested in physics you can easily skip this section.  I just want to accomplish three things:
1. Build an intuition that balancing the pole is solvable with a linear function
2. Introduce the concept of the phase space diagrams
3. Demonstrate that real world dynamic systems get really complex very quickly and are not feasible to just solve mathematically

The cartpole is an example of an equilibrium problem, where we are trying to maintain a certain position (pole being straight up).  Furthermore, this is a type of equilibrium that is known as unstable equilibrium where the slightest deviation from the target position leads to further deviation without intervention.  As the horizonal distance limit is much larger than the angle limit, we can ignore it right now in order to simplify our analysis of the environment.  The two relevant equations are 

$$\dfrac{\delta \theta}{\delta t} \approx \dot \theta$$
$(1)$

$$\dfrac{\delta \dot \theta}{\delta t} \approx \sin(\theta)$$
$(2)$

Anyone who has studied physics will know that $\sin(\theta) \approx \theta$ for small angles as this is a very popular trick in derivations.  Since the purpose of the cartpole to is maintain small angular displacement from standing straight up, we can safely assume angles will be "small" enough.  Therefore, it seems to me that we can solve an equation balancing pole with a linear model as both $\theta$ and $\dot \theta$ rely on the other in a linear relationship.  Remember that we are briefly ignoring the $x$ constraint.  To further describe the equilibrium system in this way, we can plot out a few graphs.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/PotentialEnergyGraph.png?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/PhaseSpaceDiagram.png?raw=true)
-|-
(A)|(B)

Figure A demonstrates the instability of the potential energy.  This is a good initial assessment as every system tends to move towards lowest potential energy.  Figure B is known as a phase space diagram, where we graph the rate of change in terms of both position and velocity.  As we can see, the further the angle is from zero, the greater the acceleration (rate of change of velocity).  The greater the angular velocity, the greater the rate of change of angle (obviously, but what matters it that the rate of change is pointed away from the center or equilibrium point).  While describing the same phenomena, the phase space diagram does a much better job because we can see that the velocity is much more dangerous to equilibrium than the angle (this is because the game ends at smaller angles so gravity does not get a chance to really affect the pole).  Another thing to note is that the velocity values will seem higher than possible.  This is because the game runs at $50$ fps so that any model has the ability to update the speed every $0.02$ seconds.  Therefore it is possible to achieve angular velocities greater than the $\theta$ constraint per second as there are 50 frames to adjust $\omega$ before that distance is travelled.

The dynamics of the system can be explored with the Lagrangian and the Euler-Lagrange equation.

$$\begin{equation}L = T - V\end{equation}$$
$(3)$


$$\dfrac{d}{dt}\dfrac{\delta L}{\delta\dot q} = \dfrac{\delta L}{\delta q}$$
$(4)$

Eq (3) sets the lagrangian (L) equal to the total kinetic energy of the system (T) minus the total potential energy of the system (V).  As there is no friction, the lagrangian will be a constant.  Deriving lagrangian mechanics is a little out of the scope for this project, but I'll leave links at the end for those interested.  Essentially, the Euler-Lagrange equation solves for the shortest path between two points in a given space, and feeding in the energy of a system into the equation allows us to find the shortest path through the energy space.  This gives us information as to how the dynamic system will behave..  For now, I'll keep with the assumption that solving eq (4) properly will solve the practical dynamics of the cartpole (and any similar RL environments).  The EL equation states that the derivative of the lagrange with respect to the spatial coordinates ($q$) must be equal to the derivative with respect to time of the derivative with respect to the rate of change of the spatial coordinates of the lagrange.  The kinetic and potential energies are

$$T = \frac{M\dot x^{2} + m(\dot \theta^{2}l^{2} + 2\cos(\theta)\dot \theta\dot xl + \dot x^{2})}{2}$$
$(5)$
$$V = mgl\cos(\theta)$$
$(6)$

where M is the mass of the cart, m is the mass of the pole, theta is the angle of the pole, and l is the length of the pole.  Once the inertia of the pole has been factored in as well, the nonlinear dynamics of the system becomes:

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/Nonlinear2.PNG?raw=true)
$(7)$

If this interests you, I'd highly recommend you check out the citations at the bottom of this page.  As far as this project is concerned, all I needed to demonstrate was that dynamic systems are very hard to model, even for "simple" environments.  The biggest takeaway from his derivation is that the force desired to push the cart was able to be simplified into a linear function of the four inputs.  The man who derived these equations also provided examples where the pole was balanced.

As far as this project is concerned, there are two main takeaways; deriving the dynamics of systems gets very complicated for even "simple" environments and Daniel was able to solve his version of the cartpole with a linear function of the four inputs.  Once again, if you're interested, I've linked to his derivation in my citations at the bottom.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/LagrangianBalance.gif?raw=true)


This solution differs from the traditional cartpole in a very important way.  He used physics to calculate the exact force to apply to the cart.  The cartpole problem only allows a fixed force, either in the positive or negative direction.  This means that once unstable equilibrium is reached for this environment, the force can be set to zero.  This also means that stronger force can be used to offset a poor state much faster.

The challenge that reinforcement learning is twofold; avoid having to derive the solution mathematically (often this is infeasible) and allow solutions that accomodate real world constraints (sometimes our action space is discrete).

## Heuristics

Before I build a model, the first step is to establish baseline performances in order to ground our expectations.  I played the cartpole myself and got an average around $40$.  I could have practiced and gotten better at the game but I'd like to avoid cartpole tunnel syndrome.  The Suicide heuristic is one that attempts to accelerate the pole towards the ground, ending on an average of $9$ frames.  This is the worst a model can perform and is what happens when a model only predicts one direction.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/suicide_model_sample.gif?raw=true)



Playing randomly averages around $22$ frames.  This is the best baseline, as any model that outperforms this will have at least learned something about the environment.  While playing randomly extends the duration of the game, we can see in the phase space diagram that the heuristic is incapable of reacting to the environment, allowing the instability to increase and push it away from the center.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/random_model_sample.gif?raw=true)


Controlling the angle, or accelerating to the right when the angle is positive and to the left when the angle is negative, yields an average of $41$ frames.  This heuristic is able to adapt to the environment, but quickly gains energy and momentum that spirals the pole away from the center.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/controlled_angle_sample.gif?raw=true)

Because the angular velocity is more volatile than the angle itself, it's no surprise that controlling the velocity yields an average of $200$ frames.  On an initial glance, it may seem that the heuristic works perfectly well and that the only issue is the $x$ constraint.  However, the phase space diagram exposes the fact that the angle is slowly increasing as well.  This is because the greater the angle, the more the pole tends to increase speed in that direction.  Merely controlling the speed will then cause the pole to slip as the average speed will be slightly in favor of increasing towards the pole's fall.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/controlled_velocity_sample.gif?raw=true)

## Reinforcement Learning

Now, before we can begin building models, I'd like to define a few terms and explain the concept behind reinforcement learning.  Reinforcement learning (RL) has three main components; environment, agent, and reward system.  The environment is everything that makes up the game or task that we are trying to solve.  The agent is a model that attempts to interact with the environment in order to maximize the reward.  The interplay between agent and environment forms the core of RL and looks like this:

1: <b>Initial Game State</b>:  The environment is in state $s_{i}$.  All the important information about the environment should be encoded into this game state, which the agent's model treats as an input.  This is a very important part of designing an environment.

2: <b>Agent's Action</b>:  The agent converts the state into a policy vector with the length of $n$ (number of possible actions): 

$$\pi(s_{i}) = \pi_{i}$$  
The policy predicts the expected reward for each action given the state.  The action chosen is 

$$a_{i} = \arg\max(\pi(s_{i}))$$

or the action with the highest associated $q$ value.  Note that the $i^{th}$ value of $\pi$ can be written as 

$$\pi_{i} = Q(s_{i},a_{i})$$

3: <b>Reward</b>:  The environment receives the reward from the agent and then determines the next state, $s_{i+1}$.  This does NOT have to be deterministic.  The environment also uses the reward system to give a reward to the agent.

4: <b>Update</b>:  The agent then updates the predicted $q$ value for that given state-action pair.  This is done using the bellman equation: 

$$ Q(s_{i},a_{i}) = r_i + \gamma\max(\pi(s_{i+1}))$$  

This equation means that the agent wants to estimate the value of a given action to be the reward received as well as the best possible reward from the next state. 

The bellman equation is very powerful, as it allows the agent to consider future values as well.  Even though it only directly considers states $i$ and $i+1$, the value of $\pi(s_{i+1})$ will recursively consider the next state and the one after that and so on.  In order to prevent the $q$ values from blowing up, a discount factor, $\gamma$, is used.  Gamma is a number between $0$ and $1$, and the closer to $1$ the further into the future the agent will look.  This will also have the effect of smoothing over $q$ values and will dampen the impact of each individual action.

Agents begin training with little agency.  Tracked by the value $\epsilon$, agents begin by picking moves randomly.  This helps the model get a feel for the true $q$ values and prevents the agent from getting trapped into picking the same action repeatedly.  It is also better to initialize models so that they overestimate the $q$ values.  This means that as actions are picked, their $q$ values are slowly dropped down to realistic values.  Even if an agent gains agency too quickly, they'll be picking the least chosen actions and dropping their expected $q$ values further down.  Another way around this is to allow the model to be initialized around $0$, and then set the rewards to small values and the punishment to a large negative number.  

## Markov Agent

A markov agent is the simplest RL model possible.  It treats each state as independent of the other states and determines the best moves using a Q-value function $Q(s_i,a_i)$.  In fact, this model relies on something known as the markov assumption, which assumes that the previous states have no impact on the optimal move for a given state.  The agent still cares about the success of future states when calculating the predicted $q$, it just does not need to know about the states that came before.  This applies to the cartpole because we have the speed and angular velocity of the cart and pole respectively.  However, if the agent took a screenshot of the frame as an input, this condition would not be met.  The agent would not know the speed of the cart or pole, which is very important in determining the optimal action.

We use the bellman equation to update the $q$ values for each state-action pair.  I've cheated a little bit where I calculate the true reward from the next state rather than the predicted max reward as this will be a lot more stable during training.  Since we discretize the states, there is no underlying model predicting the reward..  Instead, each time we visit a state, the agent checks to see if it has been in that position before.  If not, the $q$ values are estimated with a reward of $100$ with $5$ times visiting the state.  As stated above, the $q$ values are overestimated and I've even added an extra weight of $5$ previous visits in order to slow down the updates to the estimates.  I've included my code for the markov agent below.

In [ ]:
import numpy as np
import gym

class MarkovAgent():
    
    def __init__(self):
        
        self.memory = []
        self.Qs = {}
        self.Ns = {}
        self.done = False
        self.gamma = 0.9
        self.epsilon = 1
        self.decay = 0.99
        
        self.session_score = 0
        self.max_distance = 0
        
        self.env = gym.make("CartPole-v1")
        
    def reset(self):
        
        self.done = False
        self.memory = []
        
    def play_session(self):
        
        observation = self.env.reset()
        
        self.session_score = 0
        self.max_distance = 0
        
        while not self.done:
            
            s,action = self.take_action(observation)
            observation, reward, self.done, info = self.env.step(action)
            self.session_score += 1            
            self.memory.append([s,-100 if self.done and self.session_score<500 else reward,action])
            
            distance = observation[0]
            
            if abs(distance)>self.max_distance:
                self.max_distance = abs(distance)
            
        past_reward = 0
        
        for s, reward, action in reversed(self.memory):
            
            self.Qs[s][action] = (self.Qs[s][action]*self.Ns[s][action] + (reward + self.gamma*past_reward))/(self.Ns[s][action]+1)
            self.Ns[s][action] += 1
            past_reward = (reward + self.gamma*past_reward)
            
        self.epsilon = max(self.decay*self.epsilon,0.05)
        self.reset()
        
        
    def take_action(self,observation):
        
        s = np.asarray([round(observation[i],1+int(i/2)) for i in range(len(observation))]).tostring()
        if s not in self.Ns:
            
            self.Ns[s] = [5,5]
            self.Qs[s] = [100,100]
            
        q_list = self.Qs[s]
        
        if np.random.uniform(0,1)>self.epsilon:
            
            action = np.argmax(q_list)
            
        else:
            
            action = self.env.action_space.sample()
            
        return s,action

I round off the input values of the states in order to limit the number of unique states we have to keep track of.  Despite this, there are still $14\,641$ ($11^4$) unique initial states and over $5\,760\,000$ total states.  As you can imagine, this is a very inefficient method of training a model.  However, this does a good enough job to establish a baseline for future models.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/markovagenttraining.png?raw=true)


In order to clean up the noise, I've grouped up batches of $100$ games and plotted their means.  I refer to each batch of $100$ games as an epoch, as that matches terminology used for training actual models.  As we can see, the training slows down as it approaches $400$, and there's not a single epoch where the mean is $500$.  Just looking at the mean scores tells me nothing about what's happening, so let me show a different graph.  Each session has only three conditions where the game ends: the pole falls over, the cart leaves the frame, or the time runs out.  Since the pole falling over is the most common by far, I've decided to graph the total score and max distance of each attempt by the agent.  The latest $1\,000$ attempts are red while all other past attempts are blue.


![Original Image](graphics/training_markov.gif?raw=true)


There are two initial points of interest with the way the agent learns.  The first is that states that start off stable tend to remain stable so the agent learns how to keep the pole up (the max distance is really low).  The other is that when the pole is close to its limit, the optimal option is much more obvious (counteract the pole falling).  This leads to a high max distance but a clear path gets formed in the graph.  Eventually, the agent gets better at balancing the pole and an average velocity seems to form.  Most of the games end with $500$ points, but the agent never manages to get all of the games to max points.  While I could have continued to train longer, I had already trained for two days and felt that the marginal improvements would not be worth it.  Finally, this agent never reaches the $x$ limit.  This is because balancing the pole means that the cart will not move much, plus as the cart moves further out, it reaches states less visited.  This means that the pole balancing itself will also be much worse as each state is treated as its own entity.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/HistogramMarkov.png?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/HistogramMarkov2.png?raw=true)
-|-
A|B


We can consider the skill of the agent at any given state as a function of the number of times visited.  The $x$ values for states visited is densely focused near the center.  The $\theta$ values follow a bimodal distribution where the pole is usually slightly off center (probably because that is much easier to balance than being straight up) and is overall much more widely distributed than the $x$ values.  We can also see the effect of the larger angles making it easier for the agent to learn.  The pole spends a lot more time on the limit of its $\theta$ values than only half fallen.

I've trained the agent for long enough so that it performs well.  Because the cart does not go anywhere near the $x$ constraint, I've decided to evaluate the decision boundary based on $\theta$ and $\omega$.  Additionally, I took every state where the least chosen action was at least $300$ and trained a support vector machine on this "labelled" data.  We can see below that the agent still has a lot of noise that the SVM cleans up.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/markov_decisionboundary.png?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/svm_decisionboundary.png?raw=true)
-|-
A|B


Quick aside, a support vector machine is a very basic classification model that assumes it's possible to create a hyperplane (in the case of my model it's a straight line) that divides the two classes.  There's a lot more nuance and complexity to the models, but for the sake of this report we can consider an SVM to be a line that divides the two classes.  Evaluating the SVM on the cartpole leads to some very surprising results.  In order to make sure that I wasn't seeing things, I increased the frame limit to $1\,000$ and the model still performs perfectly.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/svm_model_sample.gif?raw=true)


Not only can the cartpole be solved with a linear decision boundary, but it is capable of playing forever while only considering two of the four variables.

## Linear Model

Next, I started to figure out how I could take what I've learned from the previous section and speed up the process by cutting out the markov agent from the process.  The equation for a SVM can be simplified into

$$\pi = \sum_{1}^{n}a_{i}x_{i}+b$$
$(8)$

where $\pi$ predicts one class if its positive and the other class if negative.  Similarly, a linear model looks like 

$$y = \sum_{1}^{n}a_{i}x_{i}+b$$
$(9)$

which is the regression form of SVM.  The key distinction between the two is that an SVM requires classified data.  The SVM needed the markov agent to label the data for it, while a linear model can learn by interacting with the environment.  The other, more subtle, difference is that the SVM just needs to learn a decision boundary, while the linear model will be required to predict $q$ values.  The Q-value function is nonlinear by nature because it includes both the reward as well as the discounted reward from the next state.  The expected reward actually balances above the true reward each frame gets, which is an interesting artifact of the Bellman equation.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/rewardfunction.png?raw=true)

I've chosen a discount factor of $0.99$ for two reasons.  I want to help the linear model learn the $q$ values by bending them into an almost linear relationship with frames.  Even though the model is not taking in frames directly as an input, this decay will smooth out the values over the different possible states as well.  Secondly, an ideal game lasts $500$ frames.  The bigger the discount factor the longer we can stretch out the effect of losing.  This means that frames earlier on in the sequence will be able to adjust to learn from the single failure.  Otherwise, most training instances will not learn anything as the reward system constantly gives out a value of $1$ until the inevitable $-100$.

In [ ]:
import numpy as np
import gym
from collections import deque


class LinearModel():
    
    def __init__(self,input_dim,output_dim):
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.matrix = np.random.rand(self.input_dim,self.output_dim)
        self.bias = np.random.uniform(-1,1)
        
    def predict(self,observation):
        
        return np.sum([self.matrix[i]*observation[i] for i in range(self.input_dim)],axis=0) + self.bias
    
    def update(self,instance,lr):
        
        state = instance[:4]
        action = instance[4]
        reward = instance[5]
        
        forward_feed = [self.matrix[i][action]*state[i] for i in range(self.input_dim)]
        error = reward - (np.sum(forward_feed) + self.bias)
        adjustment = error*lr
        
        self.bias += adjustment
        self.matrix[:,action] += np.asarray(state)*adjustment


class CartpoleAgent():
    
    def __init__(self,minmem,maxmem):
        
        self.minmem = minmem
        self.maxmem = maxmem
        self.memory = deque([], maxlen=self.maxmem)
        self.epsilon = 1
        self.decay = 0.995
        self.gamma = 0.99
        self.lr = 0.001
        
        self.env = gym.make("CartPole-v1")
        self.model = LinearModel(4,2)

        self.scores = []
        self.means = []
        self.losses = []

    def predict(self,observation):
        
        return self.model.predict(observation)
    
    def remember(self,batch_size):
        
        return random.sample(self.memory,batch_size)
    
    def train(self,epochs,num_games,batch_size):
        
        e = 0
        
        while e<epochs:
            
            for _ in range(num_games):
            
                done = False
                observation = self.env.reset()
                trainexamples = []
                session_score = 0

                while not done:

                    if np.random.uniform(0,1)>self.epsilon:
                        action = np.argmax(self.model.predict(observation))
                    else:                    
                        action = self.env.action_space.sample()

                    session_score += 1
                    next_state, reward, done, info = self.env.step(action)
                    trainexamples.append([observation,action,-100 if done and session_score<500 else 1,next_state])
                    observation = next_state

                self.scores.append(session_score)

                current_reward = 0
                for example in reversed(trainexamples):
                    current_reward = example[2] + self.gamma*current_reward
                    self.memory.append([example[0][0],example[0][1],example[0][2],example[0][3],example[1],current_reward])                 
                     
                    
            mean = np.mean(self.scores[-num_games:])
            print(f'Epoch {e+1} had a mean score of {mean} over {num_games} games.')
            self.means.append(mean)        
            
            self.epsilon = max(0.05,self.decay*self.epsilon)
            
            if len(self.memory)>self.minmem:
                
                batch = self.remember(batch_size)
                for b in batch:
                    self.model.update(b,self.lr)
                    
                e += 1

Training the model differs slightly from the markov agent as it does not update its values continuously.  Instead, the linear model plays for $100$ sessions, storing moves and results into memory, before sampling from that memory and training on that.  Instead of storing each unique combination of states as their own entity, the linear model assumes that there is a linear relationship between the four values.  This model also takes into account all four inputs, rather than just $\theta$ and $\omega$.  Finally, due an initial bug in the code, I stumbled upon something very interesting.  Normally, an agent slowly gains agency as epsilon decreases, allowing it to take control of what moves to make.  However, I accidentally trained a linear model on memory gathered from a heuristic playing completely randomly.  I've put the best performed results from both methods below.  For the model with agency, I've also included the percent chance that the agent chooses the move.  The epoch where the agent performed best had a value of $50\%$.




![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/LinearModelTrainingwAgency.png?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/LinearModelTrainingwoAgency.png?raw=true)
-|-
![Original Image](graphics/linear_model_best.gif?raw=true)|![Original Image](graphics/linear_model_random.gif?raw=true)

The first model, learns much faster than the markov agent up until a certain point.  The model, predictively, first learns how to balance the pole and focuses most of its efforts on the angular velocity.  However, the model gets to a point where it is not able to balance the pole perfectly but is still good enough to last until the cart reaches the $x$ limit.  This is a massive issue as the solution to this is unintuitive.  The model needs to accelerate towards the edge so that the pole points in the other direction.  This will allow the cart to be safely guided back towards the center.  The agent is not capable of understanding that if it ignored the edge and continued to focus on balancing the pole, it would eventually never reach the edge.  Due to the linear assumptions built into the model, any adjustment at the edge of the frame has a direct impact on how the model performs at the center.  This causes the bizarre and cyclical behaviour that we see in the performance progression of the model with agency.

The second model surprised me even more.  It is possible to train a linear model by watching a random heuristic fail at the cartpole and to then have this model perform perfectly.  Because of the simplicity of the game, randomness is good enough to build a stochastic model of good and bad moves.  The markov assumption holds, so each frame is independent of the others.  Therefore, the better move is one that puts the pole into a state further from falling down.  The randomness of the model means that the agent's memory is essentially a probabilistic function that has experimentally mapped out proximity to falling over.  

This tells me that there are two solutions to the cartpole.  The first is obvious; the model can learn how to balance the pole while adjusting the cart speed so that it remains within frame.  The second method involves balancing the pole perfectly so that the edges of the frames don't matter.  The second is a global maximum strategy unfortunately gated by the $x$ condition.  Any real model that starts from scratch won't be able to balance the pole perfectly as it will need to learn how to manage the cart's position long before it will get close to doing so.  However, we can train a linear model to perform the second method as long as we train it on a statistical distribution, either generated by a random model or markov agent.

## Neural Network

There are enough tutorials out there on how to build a DQN that I've decided to spare some space and not paste the code for that.  I will, however, include the hyperparameters in case anyone is interested:
1. <b>Layers</b>:  $4 \rightarrow 16 \rightarrow 32 \rightarrow 16 \rightarrow 2$
2. <b>Activation Function</b>:  RELU
3. <b>Learning Rate</b>:  $0.01$
4. <b>Max Epsilon</b>:  $1$
5. <b>Min Epsilon</b>:  $0.05$
6. <b>Epsilon Decay</b>:  $0.995$
7. <b>Discount Factor</b>:  $0.95$
8. <b>Optimizer</b>: Adam


Mathematically, this neural network can be simplified into the linear model if I did not include the activation functions.  The RELU activation function itself is linear, except that all negative values are converted into zero.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/relu.png?raw=true)

While this may not seem like adding much to the complexity of our model, it enables the neural network to handle the nonlinearities that confused the linear model.  In fact, here is the result from the neural network training session.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/NeuralNetworkTrain.png?raw=true)

The game is considered solved after $100$ sessions at maximum score.  Since each data point contains $100$ games played, the first value of $500$ would have ended any further training.  I kept with the training duration I used for the linear model and I'm glad I did as we get a very interesting shape.  I've divided up the session into four unique regions; A, B, C, and D.

![Original Image](graphics/neural_network_model_a.gif?raw=true)

Here is a sample from the first peak from region A.  As expected, the neural network begins to struggle when the model is good enough to balance the pole long enough to reach the $x$ limit.

![Original Image](graphics/neural_network_model_b.gif?raw=true)

Region B is where the network has gotten as good as it can get with the first approach to the cartpole.  It is capable of balancing the pole well enough that it is able to avoid the $x$ limit most of the time without having to change direction of the cart.  This method essentially solves the game at $500$ frames but would clearly fail if I extended the limit to $1\,000$.  The randomness of the starting state makes it really hard for this model to consistently prevent the cart from ending the game, however and there is a lot of noise near the top of the score ($485-500$).

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/neural_network_model_c.gif?raw=true)

Almost inexplicably, the training session takes a dive into region C.  Failure is happening often enough that the model starts to really consider $x$ as its own variable in determining how to play the game.  As the weights for this variable rise up to the level of $\theta$ and $\omega$, the model performs horribly.  It struggles to figure out how to prevent $x$ from increasing while also balancing the pole.

![Original Image](graphics/neural_network_model_d.gif?raw=true)

Finally, out of nowhere, the model performance shoots back up to $500$.  Not only does it climb back up, but there is no more noise.  Every single batch of $100$ games gives back a perfect mean of $500$ points.  As we can see from the example, the model has learned how to reset the $\dot x$ value without losing control of the pole.

## Conclusion

Despite its reputation for being unstructured and unruly, reinforcement learning is the process of training an agent to adapt to the underlying structure of its environment.  The physics is often too complicated to solve mathematically, but agents can get a feel for the rules through raw experience and exploration.  Markov agents are capable of doing this, but they are incredibly inefficient and exhaust memory.  For reference, here's a plot comparing the training time of the markov agent versus the neural network.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/markovvsnetwork.png?raw=true)

The markov agent never even managed to get a perfect session and had slowed its training down significantly.  In contrast, the network cycled through all four regions and ended on a perfect model before the markov agent managed to average $50$ points.  The purpose of an agent is to be able to map a model to a decision boundary that dictates the correct action.  This model provides generalizations that apply to previously unseen states that the markov agent cannot.  Clearly, the training progress of an agent is not obvious at first, given what I've seen.  This is especially true if there is a big time gap between gameover scenarios ($x$ and $\theta$ limits) where the timeframe is not long enough.  Overall, its important to have a model that is complex enough to be able to successfully approximate the "true" decision boundary.

## Future Work

I'm done with this project.  However, I'd like to list out a few things that I think would have been interesting to dive deeper into.  Honestly, if anyone wants to do any of these, let me know and I'll put a link to the github here.  Otherwise, thanks for reading this all the way through.

1. <b>Time Limit</b>:  It's clear to me that the time limit is a much bigger factor than I'd considered.  Most of people's projects involving cartpole use a much older and shorter frame limit of $200$.  My model was close to solving the game using the "just balance better" method with $500$ frames to last. I'm very confident that most cartpole models are only using this method rather than solving the cartpole in the method we would typically describe as knowing how to play.  I'd say increasing the frame limit to $1\,000$ would have drastically different results for the neural network and the other, simpler models would fail. 
2. <b>$x$ Limit</b>:  A large issue with the model training was that the $x$ limit was much bigger than the $\theta$.  What would happen if the agent had to consider the $x$ value while balancing the pole in the initial stages of training?  Would it learn how to truly play the game without ever having a region C?
3. <b>Initial States</b>:  A large part of overfitting with RL models has to do with the agent's ability to replicate scenarios from previous runs.  The small perturbances to the initial states allows for some variety, but I'd like to see that doubled.  It would force the model to be able to handle slightly more extreme cases rather than being able to balance the pole as it stands straight up from the beginning.
4. <b>CNN</b>:  I avoided using a convolutional neural network as I just wanted to explore basic reinforcement learning and felt that a fully connected network would be enough.  I was correct, but now I'm curious as to what kind of improvement a CNN would have and if it would be able to handle region C much more effectively.
5. <b>RL Algorithms</b>:  I was almost much more interested in the basics of reinforcement learning.  However, there are much more complex methods of training an agent than Q learning.  For example, I could have tried dueling DQNs, noisy DQNs, etc and then compared the training.  This would be interesting to see how these methods progress in training on the cartpole problem.

## Citations


1. Mathematical proof that the Euler-Lagrange equation finds the shortest path through a given space.(https://farside.ph.utexas.edu/teaching/336L/Fluid/node266.html)
2. Introduction to lagrangian mechanics where the Euler-Lagrange equation is used to solved dynamic systems (http://www.physicsinsights.org/lagrange_1.html)
3. Daniel Piedrahita's solution to a variation of the cartpole problem (https://danielpiedrahita.wordpress.com/portfolio/cart-pole-control/, https://github.com/danielpiedrahita/KDC-Project-2)
4. A separate project that I followed when recreating AlphaZero's success.  This is where I got the idea to use a dictionary to handle a function with states as inputs (https://web.stanford.edu/~surag/posts/alphazero.html, https://github.com/suragnair/alpha-zero-general)
5. There are even simpler ways to approach the cartpole.  This article is a really good initial exploration of some of these methods (http://kvfrans.com/simple-algoritms-for-solving-cartpole/)
6. While not directly relevant to my project, this paper does a very good job of examining the various pitfalls of reinforcement learning.  This was a very good resource for getting some perspective as to what I was running into (https://arxiv.org/pdf/1804.06893.pdf)
7. This textbook is the holy grail of reinforcement learning.  Rather than focusing on DQN, this book does a fantastic job of outlining the basic mathematics and though process behind training agents.  It's a math book with little code, but it's realy good as an introduction for anyone interested in RL. (Reinforcement Learning: An Introduction - by Sutton & Barton)